In [1]:
import pandas as pd
import numpy as np

In [89]:
import matplotlib.pyplot as plt

In [ ]:
'''
범주형 변수 month를 사용해서 value를 예측하는 문제
'''

In [176]:
import datetime
from calendar import isleap

def convert_partial_year(number):
    "연 단위 숫자에서 날짜를 계산하는 코드"
    year = int(number)
    d = datetime.timedelta(days=(number - year) * (365 + isleap(year)))
    day_one = datetime.datetime(year, 1, 1)
    date = d + day_one
    return date

df_nottem = sm.datasets.get_rdataset("nottem").data
df_nottem["date0"] = df_nottem[["time"]].applymap(convert_partial_year)
df_nottem["date"] = pd.DatetimeIndex(df_nottem["date0"]).round('60min') + datetime.timedelta(seconds=3600*24)
df_nottem["month"] = df_nottem["date"].dt.strftime("%m").astype('category')
del df_nottem["date0"], df_nottem["date"]
df_nottem.tail()

,time,value,month
235,1939.583333,61.8,08
236,1939.666667,58.2,09
237,1939.750000,46.7,10
238,1939.833333,46.6,11
239,1939.916667,37.8,12


In [ ]:
'''
방법1: 풀랭크
'''

In [170]:
for i in range(1, 13):
    df_nottem[f'm{i}'] = 0

In [171]:
for i in range(1, 13):
    if i < 10:
        df_nottem.loc[df_nottem.month == f'0{i}', f'm{i}'] = 1
    else:
        df_nottem.loc[df_nottem.month == f'{i}', f'm{i}'] = 1

In [172]:
df_nottem = df_nottem.drop(['month'],axis=1)
df_nottem.head()

,time,value,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12
0,1920.000000,40.6,1,0,0,0,0,0,0,0,0,0,0,0
1,1920.083333,40.8,0,1,0,0,0,0,0,0,0,0,0,0
2,1920.166667,44.4,0,0,1,0,0,0,0,0,0,0,0,0
3,1920.250000,46.7,0,0,0,1,0,0,0,0,0,0,0,0
4,1920.333333,54.1,0,0,0,0,1,0,0,0,0,0,0,0


In [173]:
X, y = df_nottem.drop(['value', 'time'], axis=1), df_nottem[['value']]

In [174]:
model = sm.OLS(y, X)
rlt = model.fit()

In [175]:
rlt.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  value   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.927
Method:                 Least Squares   F-statistic:                     277.3
Date:                Sun, 05 Sep 2021   Prob (F-statistic):          2.96e-125
Time:                        18:50:14   Log-Likelihood:                -535.82
No. Observations:                 240   AIC:                             1096.
Df Residuals:                     228   BIC:                             1137.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
m1            39.6950      0.518     76.691      0.000      38.675      40.715
m2            39.1900      0.518     75.716      0.000      38.170      40.210
m3            42.1950      0.518     81.521      0.000      41.175      43.215
m4            46.2900      0.518     89.433      0.000      45.270      47.310
m5            52.5600      0.518    101.547      0.000      51.540      53.580
m6            58.0400      0.518    112.134      0.000      57.020      59.060
m7            61.9000      0.518    119.592      0.000      60.880      62.920
m8            60.5200      0.518    116.926      0.000      59.500      61.540
m9            56.4800      0.518    109.120      0.000      55.460      57.500
m10           49.4950      0.518     95.625      0.000      48.475      50.515
m11           42.5800      0.518     82.265      0.000      41.560      43.600
m12           39.5300      0.518     76.373      0.000      38.510      40.550
==============================================================================
Omnibus:                        5.430   Durbin-Watson:                   1.529
Prob(Omnibus):                  0.066   Jarque-Bera (JB):                5.299
Skew:                          -0.281   Prob(JB):                       0.0707
Kurtosis:                       3.463   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
'''
방법2: 축소랭크
'''

In [179]:
X['m1'] = 1
X

,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12
0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
235,1,0,0,0,0,0,0,1,0,0,0,0
236,1,0,0,0,0,0,0,0,1,0,0,0
237,1,0,0,0,0,0,0,0,0,1,0,0
238,1,0,0,0,0,0,0,0,0,0,1,0


In [180]:
model = sm.OLS(y, X)
rlt = model.fit()
rlt.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  value   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.927
Method:                 Least Squares   F-statistic:                     277.3
Date:                Sun, 05 Sep 2021   Prob (F-statistic):          2.96e-125
Time:                        18:52:07   Log-Likelihood:                -535.82
No. Observations:                 240   AIC:                             1096.
Df Residuals:                     228   BIC:                             1137.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
m1            39.6950      0.518     76.691      0.000      38.675      40.715
m2            -0.5050      0.732     -0.690      0.491      -1.947       0.937
m3             2.5000      0.732      3.415      0.001       1.058       3.942
m4             6.5950      0.732      9.010      0.000       5.153       8.037
m5            12.8650      0.732     17.575      0.000      11.423      14.307
m6            18.3450      0.732     25.062      0.000      16.903      19.787
m7            22.2050      0.732     30.335      0.000      20.763      23.647
m8            20.8250      0.732     28.450      0.000      19.383      22.267
m9            16.7850      0.732     22.931      0.000      15.343      18.227
m10            9.8000      0.732     13.388      0.000       8.358      11.242
m11            2.8850      0.732      3.941      0.000       1.443       4.327
m12           -0.1650      0.732     -0.225      0.822      -1.607       1.277
==============================================================================
Omnibus:                        5.430   Durbin-Watson:                   1.529
Prob(Omnibus):                  0.066   Jarque-Bera (JB):                5.299
Skew:                          -0.281   Prob(JB):                       0.0707
Kurtosis:                       3.463   Cond. No.                         12.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""